In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [3]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
# base link
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
#filters
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [4]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [5]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [6]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [7]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
naep_wrong_resp.json() # uncomment to see error

JSONDecodeError: Invalid control character at: line 1 column 289 (char 288)

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [8]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [9]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [15]:
# your code here
import requests
import json
def pull_grades(query, grade):
    new_query = query.format(grade)
    try:
        # use requests to call the api
        naep_resp = requests.get(new_query)
        # get the json contents of the response
        naep_resp_j = naep_resp.json()
        # with result, turn it into a dataframe
        naep_resp_d = pd.DataFrame(naep_resp_j["result"])
        return naep_resp_d
    except Exception as e:
        return None

activity_query = ("https://www.nationsreportcard.gov/"
"Dataservice/GetAdhocData.aspx?"
"type=data&subject=writing&grade={}&"
"subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011")

pull_grades(activity_query, 4)
pull_grades(activity_query, 8)
pull_grades(activity_query, 12)



,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,1,Male,141.256978,1,0
1,2011,R3,2011,3,Grade 12,MN:MN,WRI,12,WRIRP,NP,National public,GENDER,Gender,2,Female,155.385917,1,0


# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [16]:
## get the key
API_KEY = "Z5D5SSwQhWAU_yquS-lEGPir21Kk-3jhy7kO9k_YGw7W6ojGqJOwfhJmLYa8bFJBHBE5aLPNFXwuQo6w7eGjAZN90OF2Y1V7Dg3LoEpsV_O3M8sOQ5Me3TqfQ60zZnYx"

In [19]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Houston,TX,77005"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [20]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': '4Xes12b8RqX2Yq6oCJxQ5g',
 'alias': 'sixty-vines-houston-3',
 'name': 'Sixty Vines',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/nO3MFIJvFnf04SaRJ_FY-Q/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/sixty-vines-houston-3?adjust_creative=vFnqGwby1fjzWrLZ8C6wgw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vFnqGwby1fjzWrLZ8C6wgw',
 'review_count': 1249,
 'categories': [{'alias': 'wine_bars', 'title': 'Wine Bars'},
  {'alias': 'newamerican', 'title': 'New American'}],
 'rating': 4.4,
 'coordinates': {'latitude': 29.71536, 'longitude': -95.41749945},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '2540 University Blvd',
  'address2': '',
  'address3': None,
  'city': 'Houston',
  'zip_code': '77005',
  'country': 'US',
  'state': 'TX',
  'display_address': ['2540 University Blvd', 'Houston, TX 77005']},
 'phone': '+12818008808',
 'display_phone': '(281) 800-8808',
 'distance': 285.1980979464283,
 'attribute

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes
0,4Xes12b8RqX2Yq6oCJxQ5g,sixty-vines-houston-3,Sixty Vines,https://s3-media2.fl.yelpcdn.com/bphoto/nO3MFI...,False,https://www.yelp.com/biz/sixty-vines-houston-3...,1249,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.4,"{'latitude': 29.71536, 'longitude': -95.41749945}",[delivery],$$,"{'address1': '2540 University Blvd', 'address2...",+12818008808,(281) 800-8808,285.198098,"{'business_temp_closed': None, 'menu_url': Non..."
1,96zutOGpWRVOVSvXtyJgbQ,maximo-west-university-place,Maximo,https://s3-media1.fl.yelpcdn.com/bphoto/rvz1OY...,False,https://www.yelp.com/biz/maximo-west-universit...,11,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",5.0,"{'latitude': 29.71615, 'longitude': -95.43351}",[],NaN,"{'address1': '6119 Edloe St', 'address2': None...",,,1417.321650,"{'business_temp_closed': None, 'menu_url': 'ht..."
2,hdZkB3fU_ogryY_cOL1PhQ,george-s-bistro-and-bar-houston,George’s Bistro & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/XaRa2z...,False,https://www.yelp.com/biz/george-s-bistro-and-b...,98,"[{'alias': 'newamerican', 'title': 'New Americ...",4.3,"{'latitude': 29.72510764462648, 'longitude': -...",[],NaN,"{'address1': '2715 Bissonnet St', 'address2': ...",+18322037329,(832) 203-7329,851.424657,"{'business_temp_closed': None, 'menu_url': Non..."
3,uro9pXVvpKzQx837wePtDg,kasra-persian-grill-no-title-2,Kasra Persian Grill,https://s3-media4.fl.yelpcdn.com/bphoto/yWedR_...,False,https://www.yelp.com/biz/kasra-persian-grill-n...,93,"[{'alias': 'persian', 'title': 'Persian/Irania...",4.6,"{'latitude': 29.716889, 'longitude': -95.417305}",[],NaN,"{'address1': '2514 Times Blvd', 'address2': No...",+17134932414,(713) 493-2414,177.140319,"{'business_temp_closed': None, 'menu_url': Non..."
4,VQZF4hu7faCkUaJ9RDRpeQ,local-foods-houston,Local Foods,https://s3-media4.fl.yelpcdn.com/bphoto/9_4Q7B...,False,https://www.yelp.com/biz/local-foods-houston?a...,1821,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 29.7195096809181, 'longitude': -9...",[delivery],$$,"{'address1': '2424 Dunstan Rd', 'address2': ''...",+17135217800,(713) 521-7800,330.006218,"{'business_temp_closed': None, 'menu_url': 'ht..."


In [21]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
4Xes12b8RqX2Yq6oCJxQ5g,4Xes12b8RqX2Yq6oCJxQ5g,sixty-vines-houston-3,Sixty Vines,https://s3-media2.fl.yelpcdn.com/bphoto/nO3MFI...,https://www.yelp.com/biz/sixty-vines-houston-3...,$$,+12818008808,(281) 800-8808
96zutOGpWRVOVSvXtyJgbQ,96zutOGpWRVOVSvXtyJgbQ,maximo-west-university-place,Maximo,https://s3-media1.fl.yelpcdn.com/bphoto/rvz1OY...,https://www.yelp.com/biz/maximo-west-universit...,NaN,,
hdZkB3fU_ogryY_cOL1PhQ,hdZkB3fU_ogryY_cOL1PhQ,george-s-bistro-and-bar-houston,George’s Bistro & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/XaRa2z...,https://www.yelp.com/biz/george-s-bistro-and-b...,NaN,+18322037329,(832) 203-7329
uro9pXVvpKzQx837wePtDg,uro9pXVvpKzQx837wePtDg,kasra-persian-grill-no-title-2,Kasra Persian Grill,https://s3-media4.fl.yelpcdn.com/bphoto/yWedR_...,https://www.yelp.com/biz/kasra-persian-grill-n...,NaN,+17134932414,(713) 493-2414
VQZF4hu7faCkUaJ9RDRpeQ,VQZF4hu7faCkUaJ9RDRpeQ,local-foods-houston,Local Foods,https://s3-media4.fl.yelpcdn.com/bphoto/9_4Q7B...,https://www.yelp.com/biz/local-foods-houston?a...,$$,+17135217800,(713) 521-7800
M70GJghogh5noUtvaZEb7g,M70GJghogh5noUtvaZEb7g,93-til-houston,93' Til,https://s3-media2.fl.yelpcdn.com/bphoto/qO7RMz...,https://www.yelp.com/biz/93-til-houston?adjust...,$$,+12818466405,(281) 846-6405
NNL2lQ1TgNX0gCs9ikATBA,NNL2lQ1TgNX0gCs9ikATBA,eunice-houston-2,Eunice,https://s3-media3.fl.yelpcdn.com/bphoto/27uoaF...,https://www.yelp.com/biz/eunice-houston-2?adju...,$$$,+18324911717,(832) 491-1717


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [27]:
# your code here

base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Houston,TX, 77013"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()

yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

<Response [200]>

{'id': 'iQFqyD-rcrKvnQzW5Wy-1A',
 'alias': 'geaux-cajun-houston',
 'name': 'Geaux Cajun',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/bMNqPrwrvEfla-Qh1khWhQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/geaux-cajun-houston?adjust_creative=vFnqGwby1fjzWrLZ8C6wgw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vFnqGwby1fjzWrLZ8C6wgw',
 'review_count': 39,
 'categories': [{'alias': 'seafood', 'title': 'Seafood'},
  {'alias': 'cajun', 'title': 'Cajun/Creole'}],
 'rating': 4.0,
 'coordinates': {'latitude': 29.793364, 'longitude': -95.194417},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '13018 Woodforest Blvd',
  'address2': 'Ste R',
  'address3': '',
  'city': 'Houston',
  'zip_code': '77013',
  'country': 'US',
  'state': 'TX',
  'display_address': ['13018 Woodforest Blvd', 'Ste R', 'Houston, TX 77013']},
 'phone': '+17134553655',
 'display_phone': '(713) 455-3655',
 'distance': 4635.984987959498,
 'at

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes
0,iQFqyD-rcrKvnQzW5Wy-1A,geaux-cajun-houston,Geaux Cajun,https://s3-media4.fl.yelpcdn.com/bphoto/bMNqPr...,False,https://www.yelp.com/biz/geaux-cajun-houston?a...,39,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 29.793364, 'longitude': -95.194417}","[pickup, delivery]",$$,"{'address1': '13018 Woodforest Blvd', 'address...",+17134553655,(713) 455-3655,4635.984988,"{'business_temp_closed': None, 'menu_url': Non..."
1,2xY74-wnEa2ko1w3vTYQdQ,italian-express-pizzeria-houston,Italian Express Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/QHogyc...,False,https://www.yelp.com/biz/italian-express-pizze...,77,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 29.7933278, 'longitude': -95.1941...","[pickup, delivery]",$$,"{'address1': '13018 Woodforest Blvd', 'address...",+17134515757,(713) 451-5757,4665.667332,"{'business_temp_closed': None, 'menu_url': 'ht..."
2,b5FbiKRf2jaJ601GOuVkHQ,lindas-seafood-houston,Linda's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/lBQx5Y...,False,https://www.yelp.com/biz/lindas-seafood-housto...,55,"[{'alias': 'seafood', 'title': 'Seafood'}]",4.4,"{'latitude': 29.7677, 'longitude': -95.19722}",[pickup],$,"{'address1': '13008 Market St Rd', 'address2':...",+17133303838,(713) 330-3838,5478.721162,"{'business_temp_closed': None, 'menu_url': 'ht..."
3,7lmJQGqtGDxmWx_P6yegUA,pappadeaux-seafood-kitchen-houston-26,Pappadeaux Seafood Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/_O-qTM...,False,https://www.yelp.com/biz/pappadeaux-seafood-ki...,269,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",3.6,"{'latitude': 29.7693896, 'longitude': -95.2176...",[delivery],$$,"{'address1': '12420 East Fwy', 'address2': '',...",+17138604001,(713) 860-4001,3946.016586,"{'business_temp_closed': None, 'menu_url': 'ht..."
4,SjJFHEecaVRENj7YdRiNhw,jacinto-city-cafe-jacinto-city,Jacinto City Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/-WAbGU...,False,https://www.yelp.com/biz/jacinto-city-cafe-jac...,57,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.5,"{'latitude': 29.768093, 'longitude': -95.237965}",[pickup],$,"{'address1': '10819 Market St', 'address2': ''...",+17134555640,(713) 455-5640,3321.900966,"{'business_temp_closed': None, 'menu_url': Non..."
